In [16]:
import pandas as pd 
import numpy as np 

In [28]:
chemistry = pd.read_csv('Chemistry publication record.csv')
chemistry["field"] = "chemistry"
medicine = pd.read_csv('Medicine publication record.csv', encoding='latin1')
medicine["field"] = "medicine"
physics = pd.read_csv('Physics publication record.csv', encoding='latin1')
physics["field"] = "physics"

data = pd.concat([chemistry, medicine, physics])

In [29]:
data

,Laureate ID,Laureate name,Prize year,Title,Pub year,Paper ID,DOI,Journal,Affiliation,Is prize-winning paper,field
0,20001,"stoddart, j",2016,a molecular shuttle,1991.0,1.976039e+09,10.1021/ja00013a096,journal of the american chemical society,northwestern university,YES,chemistry
1,20001,"stoddart, j",2016,chemical synthesis of nanostructures,1993.0,1.963538e+09,10.1557/PROC-330-57,mrs proceedings,northwestern university,NO,chemistry
2,20001,"stoddart, j",2016,formation and x ray crystal structure of pt h2...,1981.0,1.963552e+09,10.1039/C39810000851,journal of the chemical society chemical commu...,northwestern university,NO,chemistry
3,20001,"stoddart, j",2016,single walled carbon nanotubes under the influ...,2005.0,2.095637e+09,10.1002/smll.200400070,small,northwestern university,NO,chemistry
4,20001,"stoddart, j",2016,synthesis of medium heterocyclic rings from 6 ...,1974.0,2.095679e+09,10.1016/S0008-6215(00)82105-9,carbohydrate research,northwestern university,NO,chemistry
...,...,...,...,...,...,...,...,...,...,...,...
21499,10193,"zeeman, p",1902,The Effect of Magnetisation on the Nature of L...,1897.0,NaN,NaN,NaN,NaN,YES,physics
21500,10193,"zeeman, p",1902,The Influence of a Magnetic Field on Radiation...,1896.0,NaN,NaN,NaN,NaN,YES,physics
21501,10193,"zeeman, p",1902,zur hyperfeinstruktur des wismuts,1930.0,2.058955e+09,10.1007/BF01397521,european physical journal,NaN,NO,physics
21502,10193,"zeeman, p",1902,on the spectrum of ionised potassium in connex...,1924.0,2.013742e+09,10.1038/114352a0,nature,university of amsterdam,NO,physics


In [30]:
data[data["Is prize-winning paper"] == "YES"]["Affiliation"].value_counts()

harvard university                                      32
stanford university                                     25
rockefeller university                                  21
max planck society                                      20
california institute of technology                      19
                                                        ..
imperial chemical industries                             1
new england biolabs                                      1
university of texas health science center at houston     1
university of gothenburg                                 1
university of leeds                                      1
Name: Affiliation, Length: 153, dtype: int64

In [ ]:
from geopy.geocoders import Nominatim
from geopy.exc import GeocoderTimedOut
import json
import time

geolocator = Nominatim(user_agent="nobel_viz", timeout=10)  # Increase timeout to 10 seconds
institutions = data["Affiliation"]

geo_data = []
for inst in institutions:
    retries = 3
    for attempt in range(retries):
        try:
            location = geolocator.geocode(inst)
            if location:
                geo_data.append({
                    "affiliation": inst,
                    "latitude": location.latitude,
                    "longitude": location.longitude,
                    "city": location.raw.get("address", {}).get("city", ""),
                    "country": location.raw.get("address", {}).get("country", "")
                })
            else:
                print(f"No location found for {inst}")
            break  # Success, move to next institution
        except GeocoderTimedOut as e:
            print(f"Timeout for {inst}, attempt {attempt + 1}/{retries}: {e}")
            if attempt < retries - 1:
                time.sleep(2)  # Wait before retrying
            else:
                print(f"Failed to geocode {inst} after {retries} attempts")
        except Exception as e:
            print(f"Error for {inst}: {e}")
            break
        time.sleep(1)  # Respect rate limit

with open("geo_data.json", "w") as f:
    json.dump(geo_data, f, indent=2)